# Access Keyvault secrets from a Managed Online Endpoint

In this example we create a Keyvault, set a secret, and then retrieve the secret from a Managed Online Endpoint using the endpoint's system-assigned managed identity. By using the managed identity, the need to pass secrets as well as any other credentials in the image or deployment is avoided.

## Prerequisites: 
* The following additional Python packages should be installed: 
    * [azure-mgmt-keyvault](https://pypi.org/project/azure-mgmt-keyvault/) - Used to create a keyvault
    * [azure-keyvault](https://pypi.org/project/azure-keyvault/)- Used to set the secret and permissions

Install the prerequisites with the following code: 

In [ ]:
%pip install azure-mgmt-keyvault
%pip install azure-keyvault

## 1. Connect to Azure Machine Learning

### 1.1 Import Required Libraries

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.mgmt.keyvault import KeyVaultManagementClient
from azure.keyvault.secrets import SecretClient
from azure.mgmt.keyvault.models import (
    VaultCreateOrUpdateParameters,
    VaultProperties,
    Sku,
)
from azure.mgmt.keyvault.models import AccessPolicyEntry, Permissions, SecretPermissions
from azure.identity import DefaultAzureCredential, AzureCliCredential
import random, os, json, base64

### 1.2 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

### 1.3 Set variables

In [ ]:
rand = random.randint(0, 10000)

endpoint_name = f"endpt-moe-{rand}"
keyvault_name = f"kvexample{rand}"

### 1.4 Create an MLClient instance

In [ ]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

### 1.5 Create a Keyvault Management client

In [ ]:
keyvault_mgmt_client = KeyVaultManagementClient(
    credential=credential, subscription_id=subscription_id
)

## 2. Create an endpoint

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 3. Create a Keyvault

To set and retrieve secrets or other assets in the Keyvault, access policies need to be defined for each role. We need the Object (Principal) ID for the user or SP/managed identity that will read or write secrets.

### 3.1 Get your Object ID
The `oid` in your JWT [access token](https://learn.microsoft.com/en-us/azure/active-directory/develop/access-tokens) represents the Object ID of the current user or SP logged into the Azure CLI. 

In [ ]:
cli_credential = AzureCliCredential()
token = cli_credential.get_token("https://management.azure.com").token
user_or_sp_object_id = json.loads(base64.b64decode(token.split(".")[1] + "===")).get(
    "oid"
)

### 3.2 Define an AccessPolicy for the Endpoint
Allow the endpoint to get secrets in the Keyvault.

In [ ]:
endpoint_access_policy = AccessPolicyEntry(
    tenant_id=endpoint.identity.tenant_id,
    object_id=endpoint.identity.principal_id,
    permissions=Permissions(secrets=[SecretPermissions.GET]),
)

### 3.3 Define an AccessPolicy for the current user
Allow all secret permissions for the current user or Service Principal.

In [ ]:
user_or_sp_access_policy = AccessPolicyEntry(
    tenant_id=endpoint.identity.tenant_id,
    object_id=user_or_sp_object_id,
    permissions=Permissions(secrets=[SecretPermissions.ALL]),
)

### 3.5 Create the Keyvault

In [ ]:
keyvault = keyvault_mgmt_client.vaults.begin_create_or_update(
    vault_name=keyvault_name,
    resource_group_name=resource_group,
    parameters=VaultCreateOrUpdateParameters(
        location=endpoint.location,
        properties=VaultProperties(
            tenant_id=endpoint.identity.tenant_id,
            sku=Sku(name="Standard", family="A"),
            access_policies=[endpoint_access_policy, user_or_sp_access_policy],
        ),
    ),
).result()

## 4. Set a Keyvault secret

In [ ]:
secret_client = SecretClient(
    credential=credential, vault_url=f"https://{keyvault_name}.vault.azure.net"
)
secret = secret_client.set_secret(name="multiplier", value=str(7))

## 5. Create a Deployment

The [scoring script](keyvault/code/score.py) uses a `ManagedIdentityCredential` to authenticate itself to the Keyvault via a `SecretClient` from the `azure-keyvault` package. No arguments are needed to instantiate the credential object when this code is executed in a deployment, because it reads the environment variables `MSI_SECRET` and `MSI_ENDPOINT` which are already present.

As part of the deployment, we will pass an environment variable called `KV_SECRET_MULTIPLIER` and give it the value `multiplier@https://<VAULT_NAME>.vault.azure.net`. The convenience function `load_secrets` looks for environment variables with `KV_SECRET` and replaces their values with the actual value of the secret from the keyvault. 

When a request is received, `input` is multiplied by our secret. 

### 5.1 Define the deployment
The environment variable `KV_SECRET_MULTIPLIER` is set to `multiplier@<KEYVAULT_URL>`. In the scoring script, this value is parsed and passed to a SecretClient to retrieve the secret from the Keyvault.

In [ ]:
deployment = ManagedOnlineDeployment(
    name="kvdep",
    endpoint_name=endpoint_name,
    model=Model(path="keyvault"),
    code_configuration=CodeConfiguration(
        code="keyvault/code", scoring_script="score.py"
    ),
    environment=Environment(
        conda_file="keyvault/env.yml",
        image="mcr.microsoft.com/azureml/minimal-py312-inference",
    ),
    environment_variables={
        "KV_SECRET_MULTIPLIER": f"multiplier@https://{keyvault_name}.vault.azure.net"
    },
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

### 5.2 Create the deployment

In [ ]:
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

### 5.3 Update endpoint traffic

In [ ]:
endpoint.traffic = {"kvdep": 100}
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 6. Test the endpoint
The endpoint returns the value of `input` multiplied by the secret.

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name, request_file="keyvault/sample_request.json"
)

## 7. Delete assets

### 7.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)

### 7.2 Delete the keyvault

In [ ]:
keyvault_mgmt_client.vaults.delete(
    resource_group_name=resource_group, vault_name=keyvault_name
)